In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.nn.pool import global_mean_pool

In [2]:
class GCNConvByHand(nn.Module):
    """maps D x N to D x N"""
    def __init__(self, dim_in):
        super().__init__()
        self.linear = torch.nn.Linear(dim_in, dim_in, bias=True)

    def forward(self, x, A):
        num_nodes = A.shape[0]
        omega_k = self.linear.weight
        beta_k = self.linear.bias.reshape(-1, 1)
        # print("omega_k", omega_k, "beta_k", beta_k)
        H_k = x
        x = torch.matmul(
            beta_k, torch.reshape(torch.ones(num_nodes), (1, -1))
        ) + torch.matmul(omega_k, torch.matmul(H_k, A + torch.eye(num_nodes)))
        return x

In [ ]:
class GCNByHand(nn.Module):
    def __init__(self, N, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConvByHand(in_channels)
        self.conv2 = GCNConvByHand(in_channels)
        self.out = torch.nn.Linear(N, out_channels, bias=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        # print("x after first layer\n", x)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        # print("x after second layer\n", x)
        x = torch.relu(x)
        # x = torch.transpose(x, 0, 1)
        # print(x)
        # print(self.out.bias, self.out.weight, self.out.weight.shape)
        x = self.out(x)
        # print(x)
        x = torch.sigmoid(x)

        # print("x after output layer\n", x)
        # x = global_mean_pool(x, torch.zeros(x.size(1)).long())  # all are from graph 0, single graph
        return x

In [6]:
num_nodes = 3       # N
num_features = 2    # D
num_labels = 1      # O
batch_size = 10

# the following needs to be replaced by some node embeddings
x = torch.randn((batch_size, num_features, num_nodes)) # D x N

# print("x", x, x.shape)

A = torch.tensor([[0, 1, 1], [1, 0, 0], [1, 0, 0]])  # adjacency matrix ( N x N )

model = GCNByHand(N=num_nodes, in_channels=num_features, out_channels=num_labels)

out = model(x, A)   # D x O

print("output", out.shape)

output torch.Size([10, 2, 1])


A = np.array(A)

H_k = np.array(x)
# H_k

# H_k__A = H_k @ A
# H_k__A, H_k__A.shape

omega_0 = np.array(model.conv1.linear.weight.detach().numpy())
beta_0 = np.array(model.conv1.linear.bias.detach().numpy()).reshape((-1, 1))
# print(omega_0, beta_0)


preactivation = beta_0 @ np.ones(num_nodes).reshape((1, -1)) + omega_0 @ H_k @ (
    A + np.identity(num_nodes)
)
# preactivation

out_wt = np.array(model.out.weight.detach().numpy())
out_bias = np.array(model.out.bias.detach().numpy())
preactivation @ out_wt.transpose() + out_bias